# Analysis of the returns of bitcoin

## Setup

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get all dfs
def get_df(csv_basename: str, value_column: str) -> pd.DataFrame:
    # Get df from CSV and set date as index
    df = pd.read_csv(f"../data/{csv_basename}.csv", index_col="date", parse_dates=True)
    
    # Get OHLC average as value
    df[value_column] = df[["open", "high", "low", "close"]].mean(axis=1)
    df.drop(columns=["open", "high", "low", "close"], inplace=True)

    return df

df_btc = get_df("BTC", "price")
df_sp500 = get_df("SP500", "price")
df_us10y = get_df("US10Y", "yield")
df_au = get_df("AU", "price")
df_wti = get_df("WTI", "price")
df_chf = get_df("CHF", "rate")

## Returns of bitcoin across time

In [ ]:
# Get daily price change and moving averages
df_btc["price_change"] = df_btc["price"].pct_change()
df_btc["price_change_log"] = np.log(df_btc["price"] / df_btc["price"].shift(1))
df_btc["price_change_log_90d_ma"] = df_btc["price_change_log"].rolling(window=90).mean()
df_btc["price_change_log_1y_ma"] = df_btc["price_change_log"].rolling(window=365).mean()

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_btc, x=df_btc.index, y="price_change_log_90d_ma", label="Moving average (90-days)", color="#00f8ff", linewidth=0.75)
sns.lineplot(data=df_btc, x=df_btc.index, y="price_change_log_1y_ma", label="Moving average (1-year)", color="#ff5b00", linewidth=1)
plt.axhline(y=0, color="red", linewidth=1, linestyle="--")

plt.title("Daily returns of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

save_chart_as_png("3_BTC_returns")

In [ ]:
# Highest daily return
df_btc.loc[[df_btc["price_change"].idxmax()]]

In [ ]:
# Lowest daily return
df_btc.loc[[df_btc["price_change"].idxmin()]]

In [ ]:
# Total cumulative returns (which is the exact same as price appreciation since first entry)
(df_btc["price_change"] + 1).prod() - 1

In [ ]:
# Compound annual growth rate (CAGR)
start_value = df_btc["price"].iloc[0]
end_value = df_btc["price"].iloc[-1]
n = (df_btc.index[-1] - df_btc.index[0]).days / 365.25

(end_value / start_value) ** (1 / n) - 1

**Key takeaways:**
- ...

### Comparison of returns with other assets

In [ ]:
# Resample all dfs to weekly frequency, to be able to compare with bitcoin's 365 trading days
df_btc_w = df_btc[["price"]].resample("W").mean()
df_sp500_w = df_sp500.resample("W").mean()
df_us10y_w = df_us10y.resample("W").mean()
df_au_w = df_au.resample("W").mean()
df_wti_w = df_wti.resample("W").mean()
df_chf_w = df_chf.resample("W").mean()

In [ ]:
# Get 1-year moving average of returns for each weekly df
for df in [df_btc_w, df_sp500_w, df_us10y_w, df_wti_w, df_au_w, df_chf_w]:
    # Get arithmetic returns using the first column, instead of log returns since the oil data includes non-positive prices
    df["returns"] = df.iloc[:, 0].pct_change()
    # Get 1-year moving average of weekly returns
    df["returns_1y_ma"] = df["returns"].rolling(window=52).mean()

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_btc_w, x=df_btc_w.index, y="returns_1y_ma", label="Bitcoin", color="orange", linewidth=1)
sns.lineplot(data=df_sp500_w, x=df_sp500_w.index, y="returns_1y_ma", label="S&P 500", color="crimson", linewidth=1)
sns.lineplot(data=df_us10y_w, x=df_us10y_w.index, y="returns_1y_ma", label="US 10-year yield", color="dodgerblue", linewidth=1)
sns.lineplot(data=df_au_w, x=df_au_w.index, y="returns_1y_ma", label="Gold futures", color="gold", linewidth=1)
sns.lineplot(data=df_wti_w, x=df_wti_w.index, y="returns_1y_ma", label="Crude oil futures", color="limegreen", linewidth=1)
sns.lineplot(data=df_chf_w, x=df_chf_w.index, y="returns_1y_ma", label="USD/CHF", color="mediumorchid", linewidth=1)
plt.axhline(y=0, color="red", linewidth=1, linestyle="--")

plt.title("Comparison of the 1-year average of weekly returns of bitcoin with other assets across time")
plt.xlabel("")
plt.ylabel("")

#plt.yscale("symlog")
plt.ylim([-0.035, 0.14])

save_chart_as_png("3_BTC_returns_comparison")

In [ ]:
# Create table with average weekly returns of each asset for different timeframes
def get_timeframes_avg_returns(df: pd.DataFrame) -> dict[str, float]:
    return {
        "Total avg weekly returns": round(df["returns"].mean(), 4),
        "2011-2016 avg": round(df[df.index.year < 2016]["returns"].mean(), 4),
        "2016-2020 avg": round(df[(df.index.year >= 2016) & (df.index.year < 2020)]["returns"].mean(), 4),
        "2020-2024 avg": round(df[df.index.year >= 2020]["returns"].mean(), 4),
    }

df_avg_returns = pd.DataFrame({
    "Bitcoin": get_timeframes_avg_returns(df_btc_w),
    "S&P 500": get_timeframes_avg_returns(df_sp500_w),
    "US 10-year yield": get_timeframes_avg_returns(df_us10y_w),
    "Gold futures": get_timeframes_avg_returns(df_au_w),
    "Crude oil futures": get_timeframes_avg_returns(df_wti_w),
    "USD/CHF": get_timeframes_avg_returns(df_chf_w),
}).T

df_avg_returns.sort_values(by=df_avg_returns.columns[0], ascending=False)

**Key takeaways:**
- ...

### Distribution of returns

In [ ]:
plt.figure(figsize=(10, 6))

sym_limit = 0.15
df_btc_lim = df_btc.query(f"{-sym_limit} <= price_change_log <= {sym_limit}")

sns.histplot(df_btc_lim["price_change_log"], binwidth=0.01, binrange=(-sym_limit, sym_limit), color="turquoise", edgecolor="white", alpha=0.75)

plt.xlim(-sym_limit, sym_limit)

plt.title("Distribution of the daily returns of bitcoin")
plt.xlabel("Price change")
plt.ylabel("Frequency")

save_chart_as_png("3_BTC_returns_dist")

In [ ]:
# Create table with daily returns stats
pd.DataFrame({
    "Average return": [round(df_btc["price_change_log"].mean(), 4)],
    "Median return": [round(df_btc["price_change_log"].median(), 4)],
    "Standard deviation": [round(df_btc["price_change_log"].std(), 4)],
    "Min return": [round(df_btc["price_change_log"].min(), 4)],
    "Max return": [round(df_btc["price_change_log"].max(), 4)],
    "Skewness": [round(df_btc["price_change_log"].skew(), 4)],
    "Kurtosis": [round(df_btc["price_change_log"].kurt(), 4)],
})

**Key takeaways:**
- ...

#### Distribution of yearly rolling returns

In [ ]:
# Get yearly rolling returns
btc_yearly_rolling_returns = df_btc["price_change_log"].rolling(window=365).sum().dropna()

In [ ]:
plt.figure(figsize=(10, 6))

sns.histplot(btc_yearly_rolling_returns, binwidth=1/3, binrange=(-2, 6), color="purple", edgecolor="white", alpha=0.75)

plt.title("Distribution of the yearly rolling returns of bitcoin")
plt.xlabel("Price change")
plt.ylabel("Frequency")

plt.xlim(-2, 6)

save_chart_as_png("3_BTC_returns_y_rol_dist")

In [ ]:
# Create table with yearly rolling returns stats
pd.DataFrame({
    "Average return": [round(btc_yearly_rolling_returns.mean(), 4)],
    "Median return": [round(btc_yearly_rolling_returns.median(), 4)],
    "Standard deviation": [round(btc_yearly_rolling_returns.std(), 4)],
    "Min return": [round(btc_yearly_rolling_returns.min(), 4)],
    "Max return": [round(btc_yearly_rolling_returns.max(), 4)],
    "Skewness": [round(btc_yearly_rolling_returns.skew(), 4)],
    "Kurtosis": [round(btc_yearly_rolling_returns.kurt(), 4)],
})

**Key takeaways:**
- ...

## Year-over-year returns across time

In [ ]:
# Get YoY returns
# Get yearly bitcoin price df with first and last prices
df_btc_yearly = df_btc.groupby(df_btc.index.year)["price"].agg(
    first_price="first",
    last_price="last",
)
# Get YoY return
df_btc_yearly["price_change"] = (df_btc_yearly["last_price"] - df_btc_yearly["first_price"]) / df_btc_yearly["first_price"]

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly, x=df_btc_yearly.index, y="price_change", color="#f7931a")

plt.title("Year-over-year returns of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

plt.yscale("symlog")
plt.ylim(-1.25, 75)

plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{int(x)}"))
plt.yticks([-1, 0, 1, 5, 10, 50])

save_chart_as_png("3_BTC_YoY_returns")

In [ ]:
# Highest YoY return
df_btc_yearly.loc[[df_btc_yearly["price_change"].idxmax()]]

In [ ]:
# Lowest YoY return
df_btc_yearly.loc[[df_btc_yearly["price_change"].idxmin()]]

In [ ]:
# Create table with YoY returns stats
pd.DataFrame({
    "Average YoY return": [round(df_btc_yearly["price_change"].mean(), 4)],
    "Median YoY return": [round(df_btc_yearly["price_change"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["price_change"].std(), 4)],
    "Min YoY return": [round(df_btc_yearly["price_change"].min(), 4)],
    "Max YoY return": [round(df_btc_yearly["price_change"].max(), 4)],
    "Cumulative product YoY return": [(1 + df_btc_yearly["price_change"]).cumprod().iloc[-1].round(4) - 1],
})

**Key takeaways:**
- ...

### Positive returns phases

In [ ]:
# Add positive returns phase column
df_btc_yearly["positive_returns_phase"] = df_btc_yearly.index.map(lambda year:
                                                         1 if 2010 <= year <= 2013
                                                         else 2 if 2015 <= year <= 2017
                                                         else 3 if 2019 <= year <= 2021
                                                         else 4 if 2023 <= year <= 2024
                                                         else None).astype("Int64")

In [ ]:
# Get total return, max, min, average, median, standard deviation and cumulative product YoY return per positive returns phase
df_btc_yearly.groupby("positive_returns_phase").agg(
    timeframe=("price_change", lambda x: f"{x.index.min()}-{x.index.max()}"),
    total_return=("price_change", lambda x: round((df_btc_yearly.loc[x.index[-1]]["last_price"] - df_btc_yearly.loc[x.index[0]]["first_price"]) / df_btc_yearly.loc[x.index[0]]["first_price"], 2)),
    avg_yoy_return=("price_change", lambda x: round(x.mean(), 2)),
    median_yoy_return=("price_change", lambda x: round(x.median(), 2)),
    std_yoy_return=("price_change", lambda x: round(x.std(), 2)),
    max_yoy_return=("price_change", lambda x: round(x.max(), 2)),
    min_yoy_return=("price_change", lambda x: round(x.min(), 2)),
    cumprod_yoy_return=("price_change", lambda x: round((1 + x).cumprod().iloc[-1] - 1, 2))
).T

**Key takeaways:**
- ...

## Yearly risk-adjusted returns ⚖️

### Sharpe ratio

In [ ]:
# Get the volatility of monthly returns along with the number of months on yearly df
monthly_returns = df_btc["price"].resample("ME").ffill().pct_change() # Resample to monthly (month end)
monthly_stats = monthly_returns.groupby(monthly_returns.index.year).agg(
    volatility_m=("std"),
    num_months=("count"),
)
df_btc_yearly[["volatility_m", "num_months"]] = monthly_stats.reindex(df_btc_yearly.index)

In [ ]:
# Get risk free rate for each year from the 10-year US treasury yield average
df_btc_yearly["us10_yield_avg"] = df_us10y["yield"].groupby(df_us10y.index.year).mean()

In [ ]:
# Calculate Sharpe ratio, except for incomplete years (2010 and 2024)
df_btc_yearly.loc[df_btc_yearly["num_months"] == 12, "sharpe_ratio"] = round((df_btc_yearly["price_change"] - df_btc_yearly["us10_yield_avg"]) / df_btc_yearly["volatility_m"], 3)

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly[df_btc_yearly["sharpe_ratio"].notnull()], x="date", y="sharpe_ratio", color="lightblue")

plt.title("Yearly Sharpe ratio of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

plt.yticks([-15, -5, 0, 5, 10, 40])

save_chart_as_png("3_BTC_yearly_sharpe")

In [ ]:
# Highest yearly Sharpe ratio
df_btc_yearly.loc[[df_btc_yearly["sharpe_ratio"].idxmax()]]

In [ ]:
# Lowest yearly Sharpe ratio
df_btc_yearly.loc[[df_btc_yearly["sharpe_ratio"].idxmin()]]

In [ ]:
# Create table with yearly Sharpe ratios stats
pd.DataFrame({
    "Average yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].mean(), 4)],
    "Median yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["sharpe_ratio"].std(), 4)],
    "Min yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].min(), 4)],
    "Max yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].max(), 4)],
})

**Key takeaways:**
- ...

### Sortino ratio

In [ ]:
# Get the volatility of monthly negative returns on yearly df
downside_monthly_returns = monthly_returns[monthly_returns < 0]
downside_monthly_volatility = downside_monthly_returns.groupby(downside_monthly_returns.index.year).std()
df_btc_yearly["downside_volatility_m"] = downside_monthly_volatility.reindex(df_btc_yearly.index)

In [ ]:
# Calculate Sortino ratio, except for incomplete years (2010 and 2024)
df_btc_yearly.loc[df_btc_yearly["num_months"] == 12, "sortino_ratio"] = round((df_btc_yearly["price_change"] - df_btc_yearly["us10_yield_avg"]) / df_btc_yearly["downside_volatility_m"], 3)

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly[df_btc_yearly["sortino_ratio"].notnull()], x="date", y="sortino_ratio", color="lime")

plt.title("Yearly Sortino ratio of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

#plt.yscale("symlog")

plt.yticks([-100, -25, 0, 25, 100, 200, 350])

save_chart_as_png("3_BTC_yearly_sortino")

In [ ]:
# Highest yearly Sortino ratio
df_btc_yearly.loc[[df_btc_yearly["sortino_ratio"].idxmax()]]

In [ ]:
# Lowest yearly Sortino ratio
df_btc_yearly.loc[[df_btc_yearly["sortino_ratio"].idxmin()]]

In [ ]:
# Create table with yearly Sortino ratios stats
pd.DataFrame({
    "Average yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].mean(), 4)],
    "Median yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["sortino_ratio"].std(), 4)],
    "Min yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].min(), 4)],
    "Max yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].max(), 4)],
})

**Key takeaways:**
- ...

## Returns before vs since 2014

In [ ]:
# Get data before and since 2014 into separate dfs
df_btc_b2014 = df_btc[df_btc.index.year < 2014].copy()
df_btc_s2014 = df_btc[df_btc.index.year >= 2014].copy()

In [ ]:
# Add day number as a column to each df
df_btc_b2014["day_number"] = range(1, len(df_btc_b2014) + 1)
df_btc_s2014["day_number"] = range(1, len(df_btc_s2014) + 1)

In [ ]:
# Ensure the first price change is set to 0 for each df (to have 100 as the first base 100 price)
df_btc_b2014.loc[df_btc_b2014.index[0], "price_change"] = 0
df_btc_s2014.loc[df_btc_s2014.index[0], "price_change"] = 0

In [ ]:
# Add a base 100 price to each df
df_btc_b2014["base_100_price"] = (1 + df_btc_b2014["price_change"]).cumprod() * 100
df_btc_s2014["base_100_price"] = (1 + df_btc_s2014["price_change"]).cumprod() * 100

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_btc_b2014, x="day_number", y="base_100_price", color="red", linewidth=0.75, label="Before 2014")
sns.lineplot(data=df_btc_s2014, x="day_number", y="base_100_price", color="cyan", linewidth=0.75, label="Since 2014")

plt.title("Price of bitcoin (base 100) over days: before vs since 2014")
plt.xlabel("")
plt.ylabel("")
plt.legend()

plt.yscale("log")

plt.xlim(0, 4000)
plt.gca().yaxis.set_major_formatter(
    FuncFormatter(lambda x, _: (
        f"{int(x)}" if x < 1000 and x.is_integer() 
        else (f"{x:.1f}" if x < 1 
        else (f"{int(x/1000)}K" if x < 1_000_000 
        else f"{int(x/1_000_000)}M"))
    ))
)

save_chart_as_png("3_BTC_price_2014")

In [ ]:
# Create table with basic stats of both timeframes and the ratio
def calculate_stats(df: pd.DataFrame) -> dict[str, float]:
    return {
        "Number of days": df["price"].count(),
        "Final price (base 100)": round(df.iloc[-1]["base_100_price"], 4),
        "Cumulative returns": round((df["price_change"] + 1).prod() - 1, 4),
    }

df_tf_stats = pd.DataFrame({
    "Before 2014": calculate_stats(df_btc_b2014),
    "Since 2014": calculate_stats(df_btc_s2014),
})

df_tf_stats["Ratio (before / since)"] = (df_tf_stats["Before 2014"] / df_tf_stats["Since 2014"]).round(4)

# Format output to remove scientific notation and make more readable
df_tf_stats = df_tf_stats.map(lambda x: f"{float(x):,.2f}" if isinstance(x, (int, float)) else x)

df_tf_stats.T

**Key takeaways:**
- ...

### Distribution of returns (before vs since 2014)

In [ ]:
plt.figure(figsize=(10, 6))

sym_limit = 0.15
df_btc_lim = df_btc.query(f"{-sym_limit} <= price_change_log <= {sym_limit}")

sns.histplot(df_btc_lim[df_btc_lim.index.year < 2014]["price_change_log"], stat="probability", binwidth=0.01, binrange=(-sym_limit, sym_limit), color="blue", alpha=0.75, label="Before 2014")
sns.histplot(df_btc_lim[df_btc_lim.index.year >= 2014]["price_change_log"], stat="probability", binwidth=0.01, binrange=(-sym_limit, sym_limit), color="green", alpha=0.66, label="Since 2014")

plt.xlim(-sym_limit, sym_limit)

plt.title("Distribution of the daily returns of bitcoin")
plt.xlabel("Price change")
plt.ylabel("Probability")
plt.legend()

save_chart_as_png("3_BTC_returns_dist_2014")

In [ ]:
# Create table with stats of both timeframes and the ratio
def calculate_stats(df: pd.DataFrame) -> dict[str, float]:
    return {
        "Average return": round(df["price_change_log"].mean(), 4),
        "Median return": round(df["price_change_log"].median(), 4),
        "Standard deviation": round(df["price_change_log"].std(), 4),
        "Min return": round(df["price_change_log"].min(), 4),
        "Max return": round(df["price_change_log"].max(), 4),
        "Skewness": round(df["price_change_log"].skew(), 4),
        "Kurtosis": round(df["price_change_log"].kurt(), 4),
    }

df_tf_stats = pd.DataFrame({
    "Before 2014": calculate_stats(df_btc_b2014),
    "Since 2014": calculate_stats(df_btc_s2014),
})

df_tf_stats["Ratio (before / since)"] = (df_tf_stats["Before 2014"] / df_tf_stats["Since 2014"]).round(4)
df_tf_stats.T

**Key takeaways:**
- ...